In [2]:
import pandas as  pd 
import numpy as np  

import pickle
import pmdarima
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss 
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from pmdarima.pipeline import Pipeline
from pmdarima.preprocessing import LogEndogTransformer, BoxCoxEndogTransformer
from pmdarima.arima import ARIMA
from datetime import datetime, timedelta

In [8]:
train=pd.read_csv('C:/PROJECTS/Predicting the AQI of Indian Cities Using Machine Learning/dataset/train.csv')
sub=pd.read_csv('C:/PROJECTS/Predicting the AQI of Indian Cities Using Machine Learning/dataset/sample_submission.csv')

In [9]:
train['Date'] = pd.to_datetime(train['Date'])

In [10]:
train['Date'] = pd.to_datetime(train['Date'])

def imputation_mean(data, column, mean):
  data[column] = data[column].fillna(mean)
  return data

cols = ['PM2.5', 'PM10', 'O3', 'CO']
for c in cols:
  mean = train[c].mean()
  data = imputation_mean(train, c, mean)

train.isnull().sum()

ID_Date      0
StateCode    0
StationId    0
Date         0
PM2.5        0
PM10         0
O3           0
CO           0
SO2          0
AQI          0
dtype: int64

In [11]:
train['Year'] = train['Date'].dt.year
train['Month'] = train['Date'].dt.month
train['Day'] = train['Date'].dt.day

Scaling Data

In [12]:
scaler = MinMaxScaler()
train[['AQI']] = scaler.fit_transform(train['AQI'].values.reshape(-1,1))
train.head()

,ID_Date,StateCode,StationId,Date,PM2.5,PM10,O3,CO,SO2,AQI,Year,Month,Day
0,AS_4_2021-08-17,AS,4,2021-08-17,51.8364,120.3702,18.0234,0.4896,14.3106,0.083247,2021,8,17
1,AS_4_2021-08-18,AS,4,2021-08-18,59.4762,87.0978,34.5270,0.4794,14.7594,0.114464,2021,8,18
2,AS_4_2021-08-19,AS,4,2021-08-19,61.0776,79.6008,42.0546,0.5202,16.7484,0.070760,2021,8,19
3,AS_4_2021-08-20,AS,4,2021-08-20,56.7222,94.2072,45.7878,0.4998,27.6114,0.096774,2021,8,20
4,AS_4_2021-08-21,AS,4,2021-08-21,85.3332,136.7820,44.5638,1.1118,26.4180,0.096774,2021,8,21


Feature engineering

In [13]:
train['State_Station']= train['StateCode'].astype(str) + "_" + train['StationId'].astype(str)
train.head()

,ID_Date,StateCode,StationId,Date,PM2.5,PM10,O3,CO,SO2,AQI,Year,Month,Day,State_Station
0,AS_4_2021-08-17,AS,4,2021-08-17,51.8364,120.3702,18.0234,0.4896,14.3106,0.083247,2021,8,17,AS_4
1,AS_4_2021-08-18,AS,4,2021-08-18,59.4762,87.0978,34.5270,0.4794,14.7594,0.114464,2021,8,18,AS_4
2,AS_4_2021-08-19,AS,4,2021-08-19,61.0776,79.6008,42.0546,0.5202,16.7484,0.070760,2021,8,19,AS_4
3,AS_4_2021-08-20,AS,4,2021-08-20,56.7222,94.2072,45.7878,0.4998,27.6114,0.096774,2021,8,20,AS_4
4,AS_4_2021-08-21,AS,4,2021-08-21,85.3332,136.7820,44.5638,1.1118,26.4180,0.096774,2021,8,21,AS_4


Check stationary

In [14]:
non_stationary_state_station = []

for state in train['StateCode'].unique():
  state_data = train[train['StateCode'] == state]
  for station in state_data['StationId'].unique():  
    state_station_data = state_data[state_data['StationId'] == station]
    result = adfuller(state_station_data['AQI'])
    
    if result[1] > 0.05:
      non_stationary_state_station.append(str(state) + "_" + str(station))

In [15]:
non_stationary_state_station = sorted(non_stationary_state_station)
non_stationary_state_station

['DL_18',
 'DL_26',
 'DL_32',
 'DL_38',
 'DL_39',
 'DL_8',
 'KA_10',
 'KA_7',
 'MH_10',
 'MH_11',
 'MH_12',
 'MH_13',
 'MH_14',
 'MH_15',
 'MH_16',
 'MH_17',
 'WB_12',
 'WB_13',
 'WB_14',
 'WB_16']

Model

In [16]:
def create_aqi_dict(data):
  aqi = dict()
  for state_station in data['State_Station'].unique():
    aqi[state_station] = data[data['State_Station'] == state_station][['AQI']]
  return aqi

# non_stationary_aqi = create_aqi_dict(non_stationary_data)
# stationary_aqi = create_aqi_dict(stationary_data)
aqi = create_aqi_dict(train)

res_bc = pd.DataFrame(columns=['ID_Date', 'AQI'])
pred_days = 28

# model_dict = dict()
# model_dict_new = dict()
model_dict_latest = dict() 
for code, d in aqi.items():
    print(f"Working on {code} ...")
    
    # data.dropna(inplace=True)

    pipeline = Pipeline([
        ("BoxCoxTransform", BoxCoxEndogTransformer(lmbda2=1e-6)),
        # ("LogTransform", LogEndogTransformer()),
        ("model", ARIMA(order = (7,1,2), trace=False, suppress_warnings=True, n_fits=50, n_jobs=-1))
    ])

    pipeline.fit(d['AQI'])
    # model = AutoARIMA(df['AQI'], start_p=7, start_q=1, d=None, max_p=10, max_q=3, max_d=1, start_P=0, start_Q=0, D=0, max_P=0, max_Q=0, max_D=0, m=1, seasonal=True, \
    #        trace=False, error_action='warn', suppress_warnings=True, random_state = 42, n_fits=40, n_jobs=-1)
    # model = auto_arima(df['AQI'], trace=False, error_action='warn', suppress_warnings=True, n_fits=40, n_jobs=-1)
    
    # model_dict_latest[code] = model

    # Serialize with Pickle
    file_path = f'C:/PROJECTS/Predicting the AQI of Indian Cities Using Machine Learning/H2o_models/arima_boxcox_{code}.pkl'
    with open(file_path, 'wb') as pkl:
      pickle.dump(pipeline, pkl)

    start_date = (train.Date.iloc[-1].date() + timedelta(days=1)).isoformat()
    end_date = (train.Date.iloc[-1].date() + timedelta(days=28)).isoformat()

    date_list = pd.date_range(start_date, end_date, freq='D')
    date_list = [code + '_' + str(day) for day in date_list]
    
    pred = pipeline.predict(n_periods=pred_days)

    #inverse predictions scaling
    yhat = scaler.inverse_transform(pred.reshape(1,-1))

    res_bc = pd.concat([res_bc, pd.DataFrame({'ID_Date': date_list, 'AQI': yhat.flatten()})])
    
res_bc.reset_index(drop=True, inplace=True)


Working on AS_4 ...
Working on DL_4 ...
Working on DL_6 ...
Working on DL_8 ...
Working on DL_12 ...
Working on DL_13 ...
Working on DL_18 ...
Working on DL_19 ...
Working on DL_21 ...
Working on DL_23 ...
Working on DL_25 ...
Working on DL_26 ...
Working on DL_27 ...
Working on DL_29 ...
Working on DL_30 ...
Working on DL_32 ...
Working on DL_35 ...
Working on DL_38 ...
Working on DL_39 ...
Working on DL_40 ...
Working on DL_41 ...
Working on KA_7 ...
Working on KA_9 ...
Working on KA_10 ...
Working on KA_11 ...
Working on KA_14 ...
Working on MH_10 ...
Working on MH_11 ...
Working on MH_12 ...
Working on MH_13 ...
Working on MH_14 ...
Working on MH_15 ...
Working on MH_16 ...
Working on MH_17 ...
Working on TN_5 ...
Working on TN_8 ...
Working on WB_12 ...
Working on WB_13 ...
Working on WB_14 ...
Working on WB_16 ...


In [19]:
res_bc.to_csv('C:/PROJECTS/Predicting the AQI of Indian Cities Using Machine Learning/output_file/submission.csv')